## 경북일보  
약 천개 기사 크롤링

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import time
import random

'''
url 읽어오기
'''
def get_html(url):
    _html = ""
    resp = requests.get(url)
    if resp.status_code == 200:
        _html = resp.text

    return _html

kyongbuk_urls=[]
kyongbuk_text=[]

search=input()    
u = "https://www.kyongbuk.co.kr/engine_yonhap/search.php?div_code=&cust_div_code=&sfield=&article_type=&others_cont_type=&period=all&from_date=&to_date=&searchword="+search+"&picktab=article&others_cont_type=&sort=date"

cn=get_html(u)
sp = BeautifulSoup(cn, 'lxml', from_encoding='utf-8')
nt=sp.find("div", {"total-list-row"}).find_all("div", {"class":"contents"})
nn=sp.find('h3').text

news_num=(nn[nn.find('과')+1:nn.find('건')])
news_num=news_num.replace(',', '')
news_num=int(news_num)
check=1

print('*** crolling start ***')

for x in range(0, int(news_num/10)+1):    #int(news_num/10)+1
    url = "https://www.kyongbuk.co.kr/engine_yonhap/search.php?page="+str(x+1)+"&total=6707&picktab=article&searchcont=article&others_cont_type=&div_code=&cust_div_code=&sfield=&article_type=&period=all&from_date=&to_date=&sort=date&searchword="+search+"&orgsearchword="
    print(url)
    collection_news=get_html(url)
    soup = BeautifulSoup(collection_news, 'html.parser')
    news_table=soup.find("div", {"total-list-row"}).find_all("div", {"class":"title"})
    print(news_table)

    for index in news_table:     
        try:
            info_news=index.find("a")
            _url=info_news["href"]
            print("new", info_news)
            print("url", _url)    
            kyongbuk_urls.append(_url)

            _select_news=get_html(_url)
            select_news=BeautifulSoup(_select_news, 'html.parser')
            _news=select_news.find("div", {"class":"user-content"}).find_all("div", {"id":"article-view-content-div"})

            for item in _news:
                _text=item.text.split(".")
                _text =str(_text)
                _text = re.sub('[0-9a-zA-Z""]', '', _text)
                _text = re.sub('[!@"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·▶◆]', '', _text) 
                kyongbuk_text.append(_text)

            time.sleep(random.uniform(0.5, 2))
            if check==news_num:
                print("xx")
                break

            print("ok", check, x)
            check+=1

        except AttributeError:
            pass
    

In [ ]:
import pandas as pd
dftext=pd.DataFrame(kyongbuk_text)
dftext.to_excel('./경북일보.xlsx', header=False, index=False)

In [ ]:
dfurl=pd.DataFrame(kyongbuk_urls)
dfurl.to_excel('./경북일보urls.xlsx', header=False, index=False)